STEP 1 — Merge National Time-Series + Monthly Capacity

In [1]:
import pandas as pd

# === 1️⃣ File paths ===
energy_file = '../data/processed/Cleaned_Energy_Data.csv'
wind_weather_file = '../data/processed/Cleaned_Weather_Data_ERA5.csv'
solar_weather_file = '../data/processed/Solar_Radiation_Monthly_Ireland.csv'
wind_capacity_file = '../data/processed/Cleaned_Wind_Capacity.csv'
solar_capacity_file = '../data/processed/Cleaned_Solar_Capacity.csv'

output_file = '../data/processed/National_TimeSeries_With_Capacity.csv'

In [2]:
# === 2️⃣ Load files ===
df_energy = pd.read_csv(energy_file)
df_wind_weather = pd.read_csv(wind_weather_file)
df_solar_weather = pd.read_csv(solar_weather_file)
df_wind_cap = pd.read_csv(wind_capacity_file)
df_solar_cap = pd.read_csv(solar_capacity_file)

In [3]:
# === 3️⃣ Standardize Dates ===
# Energy: assume already YYYY-MM-DD
df_energy['Date'] = pd.to_datetime(df_energy['Date']).dt.strftime('%Y-%m-%d')

In [4]:
# Wind Weather: drop ERA5 meta
df_wind_weather['Date'] = pd.to_datetime(df_wind_weather['Date']).dt.strftime('%Y-%m-%d')
df_wind_weather = df_wind_weather.drop(columns=['number', 'expver'], errors='ignore')

In [5]:
# Solar Radiation: fix timestamp, keep YYYY-MM-DD only
df_solar_weather['Date'] = pd.to_datetime(df_solar_weather['Date']).dt.strftime('%Y-%m-%d')

In [6]:
# Wind Capacity: already good
df_wind_cap['Date'] = pd.to_datetime(df_wind_cap['Date']).dt.strftime('%Y-%m-%d')

In [7]:
# Solar Capacity: fix DD-MM-YYYY → YYYY-MM-DD
df_solar_cap['Date'] = pd.to_datetime(df_solar_cap['Date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

In [8]:
# === 4️⃣ Merge step-by-step ===
df_merged = df_energy.merge(df_wind_weather, on='Date', how='left')
df_merged = df_merged.merge(df_solar_weather, on='Date', how='left')
df_merged = df_merged.merge(df_wind_cap, on='Date', how='left')
df_merged = df_merged.merge(df_solar_cap, on='Date', how='left')

In [9]:
# === 5️⃣ Final checks ===
print("\n✅ Final National Time-Series Shape:", df_merged.shape)
print(df_merged.head(3))


✅ Final National Time-Series Shape: (180, 9)
         Date  Wind_GWh  Solar_GWh  Wind_Speed_10m  Temperature_Celsius  \
0  2010-01-01   248.002        0.0        0.786551             4.149077   
1  2010-02-01   150.952        0.0        1.154458             4.126316   
2  2010-03-01   251.394        0.0        0.872089             6.327460   

   Cloud_Cover  Solar_Radiation_MJ_per_m2  Wind_Capacity_MW  Solar_Capacity_MW  
0     0.668563                  2971663.8          1270.807                0.6  
1     0.694220                  5765193.0          1286.107                0.6  
2     0.653995                 10048856.0          1303.657                0.6  


In [10]:
# === 6️⃣ Save output ===
df_merged.to_csv(output_file, index=False)
print(f"\n✅ STEP 1 saved: {output_file}")


✅ STEP 1 saved: ../data/processed/National_TimeSeries_With_Capacity.csv


STEP 2 — Expand Regional Capacity Table

In [18]:
import pandas as pd

# === 1️⃣ File paths ===
regional_file = '../data/processed/Regional_Capacity_Table.csv'
national_ts_file = '../data/processed/National_TimeSeries_With_Capacity.csv'
output_file = '../data/processed/Regional_Capacity_Expanded.csv'

In [19]:
# === 2️⃣ Load both ===
df_regional = pd.read_csv(regional_file)
df_national_ts = pd.read_csv(national_ts_file)

In [20]:
# === 3️⃣ Get unique Dates from national time-series ===
unique_dates = df_national_ts['Date'].unique()
print(f"✅ Number of unique months: {len(unique_dates)}")

✅ Number of unique months: 180


In [21]:
# === 4️⃣ Create expanded Regional table ===
# Add a dummy key to cross join
df_regional['key'] = 1
df_dates = pd.DataFrame({'Date': unique_dates})
df_dates['key'] = 1

In [22]:
# Cross join: each Regional row repeats for every Date
df_expanded = pd.merge(df_dates, df_regional, on='key').drop(columns=['key'])

In [23]:
# === 5️⃣ Sanity check ===
print("\n✅ Sample of expanded Regional Capacity:")
print(df_expanded.head(5))
print("\n✅ Final expanded shape:", df_expanded.shape)


✅ Sample of expanded Regional Capacity:
         Date   Zone   Type Connection_Type  Installed_MW
0  2010-01-01   East   Wind             DSO       349.389
1  2010-01-01   East   Wind             TSO       491.400
2  2010-01-01   East  Wind              TSO        34.000
3  2010-01-01  North   Wind             DSO       340.498
4  2010-01-01  North   Wind             TSO       356.900

✅ Final expanded shape: (2880, 5)


In [24]:
df_expanded['Type'] = df_expanded['Type'].str.strip()

In [25]:
# === 6️⃣ Save output ===
df_expanded.to_csv(output_file, index=False)
print(f"\n✅ STEP 2 saved: {output_file}")


✅ STEP 2 saved: ../data/processed/Regional_Capacity_Expanded.csv


STEP 3 — Final Merge: National + Regional

In [26]:
import pandas as pd

In [27]:
# === 1️⃣ File paths ===
national_ts_file = '../data/processed/National_TimeSeries_With_Capacity.csv'
regional_expanded_file = '../data/processed/Regional_Capacity_Expanded.csv'
output_file = '../data/processed/Final_Model_Ready.csv'

In [28]:
# === 2️⃣ Load both ===
df_national = pd.read_csv(national_ts_file)
df_regional = pd.read_csv(regional_expanded_file)

In [29]:
# === 3️⃣ Merge: cross join on Date
df_final = pd.merge(
    df_regional,
    df_national,
    on='Date',
    how='left'
)

In [30]:
# === 4️⃣ Sanity checks ===
print("\n✅ Final merged shape:", df_final.shape)
print("\n✅ Sample rows:")
print(df_final.head(5))


✅ Final merged shape: (2880, 13)

✅ Sample rows:
         Date   Zone  Type Connection_Type  Installed_MW  Wind_GWh  Solar_GWh  \
0  2010-01-01   East  Wind             DSO       349.389   248.002        0.0   
1  2010-01-01   East  Wind             TSO       491.400   248.002        0.0   
2  2010-01-01   East  Wind             TSO        34.000   248.002        0.0   
3  2010-01-01  North  Wind             DSO       340.498   248.002        0.0   
4  2010-01-01  North  Wind             TSO       356.900   248.002        0.0   

   Wind_Speed_10m  Temperature_Celsius  Cloud_Cover  \
0        0.786551             4.149077     0.668563   
1        0.786551             4.149077     0.668563   
2        0.786551             4.149077     0.668563   
3        0.786551             4.149077     0.668563   
4        0.786551             4.149077     0.668563   

   Solar_Radiation_MJ_per_m2  Wind_Capacity_MW  Solar_Capacity_MW  
0                  2971663.8          1270.807                0.

In [31]:
# Check unique zones and types
print("\n✅ Zones in final:", df_final['Zone'].unique())
print("✅ Types in final:", df_final['Type'].unique())


✅ Zones in final: ['East' 'North' 'South' 'West']
✅ Types in final: ['Wind' 'Solar']


In [32]:
# === 5️⃣ Save final model-ready dataset ===
df_final.to_csv(output_file, index=False)
print(f"\n✅ STEP 3 saved: {output_file}")


✅ STEP 3 saved: ../data/processed/Final_Model_Ready.csv
